In [3]:
import pandas as pd

In [29]:
sheets_dict = pd.read_excel('SI2.xlsx', sheet_name=None)


all_sheets = []
df = pd.DataFrame()
for name, sheet in sheets_dict.items():
    sheet = sheet.iloc[3:, :7]
    sheet["component"] = name
    df = pd.concat([df, sheet])
    #print()

In [30]:
df.to_excel("metals_db.xlsx")

In [7]:
from premise.activity_maps import InventorySet, get_mapping
from premise import *
import brightway2 as bw
bw.projects.set_current("new")

In [5]:
ndb = NewDatabase(
        scenarios = [
            {"model":"image", "pathway":"SSP2-Base", "year":2020,},
            #{"model":"image", "pathway":"SSP2-Base", "year":2050,},
        ],        
        source_db="ecoinvent 3.8 cutoff",
        source_version="3.8",
        key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    )

premise v.(1, 4, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [6]:
inv = InventorySet(ndb.database)

In [44]:
fp = "/Users/romain/GitHub/premise/premise/data/materials_emission_factors/materials_mapping.yml"
mapping = get_mapping(filepath=fp, var="ecoinvent_aliases")
inv.generate_sets_from_filters(mapping)

{'ELECTRO_MOTOR': {'electric motor production, for electric scooter',
  'electric motor production, vehicle',
  'electric motor production, vehicle (electric powertrain)'},
 'W_on_AG': {'wind power plant construction, 2MW, offshore, moving parts',
  'wind power plant construction, 800kW, moving parts',
  'wind turbine construction, 2MW, onshore',
  'wind turbine construction, 4.5MW, onshore',
  'wind turbine construction, 750kW, onshore'},
 'W_on_SG_E_DD': {'wind power plant construction, 2MW, offshore, moving parts',
  'wind power plant construction, 800kW, moving parts',
  'wind turbine construction, 2MW, onshore',
  'wind turbine construction, 4.5MW, onshore',
  'wind turbine construction, 750kW, onshore'},
 'W_on_HTS_DD': {'wind power plant construction, 2MW, offshore, moving parts',
  'wind power plant construction, 800kW, moving parts',
  'wind turbine construction, 2MW, onshore',
  'wind turbine construction, 4.5MW, onshore',
  'wind turbine construction, 750kW, onshore'},
 'W_o